In [1]:
import glob

import numpy as np
import modvis.ats_xdmf as xdmf
import modvis.plot_vis_file as pv
import pandas as pd
from matplotlib import pyplot as plt
import h5py
import os
import re
import datetime
from datetime import  timedelta
import pandas as pd

In [2]:
list_indices_surf_subsurface = np.load("list_indices_surf_subsurface.npy").tolist()

In [3]:
sampled_params = pd.read_csv("sampled_params.csv")
sampled_params = sampled_params[["@serial_number@","@Diff_Temp_conifer@", "@Melt_rate_conifer@"]]
sampled_params

,@serial_number@,@Diff_Temp_conifer@,@Melt_rate_conifer@
0,0,2.559871,1.634214
1,1,2.303413,2.701860
2,2,2.107107,1.888342
3,3,2.654204,2.201876
4,4,2.247950,1.667534
...,...,...,...
95,95,2.877621,2.042007
96,96,2.381550,1.880526
97,97,2.037353,2.690644
98,98,2.124129,2.244925


In [4]:
float(sampled_params.iloc[0]["@Melt_rate_conifer@"])

1.6342138101352828

In [5]:
dir = "/pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations"

In [6]:
def daymet_h5(name = 'air temperature [K]', file_path = dir + '/shared_files/watershed_daymet-ats-2010-01-01_2023-12-31.h5'):
    with h5py.File(file_path, 'r') as hdf_file:
        group_name = name # Replace 'group_name' with the actual name of the group
        group = hdf_file[group_name]

        # Get the names of all datasets within the group
        dataset_names = list(group.keys())
        dataset_names = np.array(np.sort(np.array(dataset_names,dtype = 'int64')))

        # Determine the shape of the 3D array
        num_datasets = len(dataset_names)
        dataset_shape = group[str(dataset_names[0])].shape
        array_shape = (num_datasets,) + dataset_shape

        # Initialize the 3D NumPy array
        numpy_array = np.zeros(array_shape, dtype=group[str(dataset_names[0])].dtype)

        # Fill the 3D array with dataset values
        for i, dataset_name in enumerate(dataset_names):
            numpy_array[i] = group[str(dataset_name)][:]

        return numpy_array[:, 0, 0]
    
temp = daymet_h5(name = 'air temperature [K]')
snow = daymet_h5(name = 'precipitation snow [m SWE s^-1]')
rain = daymet_h5(name = 'precipitation rain [m s^-1]')
shortwave = daymet_h5(name = 'incoming shortwave radiation [W m^-2]')
vapor_pressure = daymet_h5(name = 'vapor pressure air [Pa]')


temp = temp[3558:5019]
snow = snow[3558:5019]
rain = rain[3558:5019]
shortwave = shortwave[3558:5019]
vapor_pressure = vapor_pressure[3558:5019]

In [7]:
# share two vertices i.e. a link
def mesh_adj(visfile_surface):
    adj = np.zeros((visfile_surface.conn.shape[0],visfile_surface.conn.shape[0]))
    list_edges = []
    for i,elem_i in enumerate(visfile_surface.conn[:,1:]):
        for j,elem_j in enumerate(visfile_surface.conn[:,1:]):
            if elem_i[0] in elem_j and elem_i[1] in elem_j:
                list_edges.append((i,j))
                adj[i,j] = 1
                adj[j,i] = adj[i,j]
            elif elem_i[0] in elem_j and elem_i[2] in elem_j:
                list_edges.append((i,j))
                adj[i,j] = 1
                adj[j,i] = adj[i,j]
            elif elem_i[1] in elem_j and elem_i[2] in elem_j:
                list_edges.append((i,j))
                adj[i,j] = 1
                adj[j,i] = adj[i,j]
    return list_edges, adj

def resample(D_array, resampling_list):
    new_3D_array = np.zeros(D_array.shape)
    for n,i in enumerate(resampling_list):
        new_3D_array[:,n,:] = D_array[:,i,:]
    return new_3D_array

In [8]:
def input_output(model_dir):
    
    visfile = xdmf.VisFile(model_dir, domain=None, load_mesh=True, columnar=True,model_time_unit='d')
    visfile_S = xdmf.VisFile(model_dir, domain='surface', load_mesh=True, model_time_unit='d')
    
    
    """
    # this should be refactored
    list_indices_surf_subsurface = [] # list to hold indices of subsurface centroid in surface mesh
    for coord in visfile_S.centroids[:,:2]:
        matches = np.all(visfile.centroids[:,0,:][:,:2] == coord, axis=1)  # Compare rows
        if np.any(matches):  # Check if any row matches
            index = np.where(matches)[0][0]  
            #print(f"Element found at index {index}")
        #else:
            #print("Element not found")
        list_indices_surf_subsurface.append(int(index))
        
     """   
    sim_no = int(model_dir.split("sim")[-1])
    perm = visfile.getArray('permeability')
    poro = visfile.getArray('porosity')
    wc = visfile.getArray('water_content')
    saturation = visfile.getArray('saturation_liquid')
    
    new_perm = resample(D_array=perm, resampling_list=list_indices_surf_subsurface)
    new_poro = resample(D_array=poro, resampling_list=list_indices_surf_subsurface)
    new_wc = resample(D_array=wc, resampling_list=list_indices_surf_subsurface)
    new_saturation = resample(D_array=saturation, resampling_list=list_indices_surf_subsurface) #(1461,740,18)
    
    # refactor this as well
    node_features = []  # (740,1461,40)
    for node in range(740):
        node_feature_over_time = []
        for time in range(1461):
            node_feature = []
            node_feature.append(float(temp[time]))
            node_feature.append(float(snow[time]))
            node_feature.append(float(rain[time]))
            node_feature.append(float(shortwave[time]))
            node_feature = node_feature + new_perm[time][node].tolist() + new_poro[time][node].tolist() 
            node_feature.append(float(sampled_params.iloc[sim_no]["@Diff_Temp_conifer@"]))#@Diff_Temp_conifer@ 
            node_feature.append(float(sampled_params.iloc[sim_no]["@Melt_rate_conifer@"]))#@Melt_rate_conifer@ 	
            node_feature_over_time.append(node_feature)
        node_features.append(node_feature_over_time)
    
    node_features = np.array(node_features)
    node_features = node_features[:,731:,:]
    node_features = np.transpose(node_features, (0, 2, 1))
    new_saturation = new_saturation[731:,:,:]
    new_saturation = np.transpose(new_saturation, (1, 2, 0))
    
    return node_features, new_saturation

In [9]:
model_dir = "/pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim99"

number = int(model_dir.split("sim")[-1])
number

99

In [10]:
for index, folder in enumerate(glob.glob(dir+"/sim*")):
    print(index,folder)

0 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim99
1 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim75
2 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim88
3 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim32
4 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim64
5 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim72
6 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim50
7 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim51
8 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim73
9 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim10
10 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim52
11 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim39
12 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim41
13 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim86
14 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim65
15 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simu

In [11]:
for index, folder in enumerate(glob.glob(dir+"/sim*")):
    print(index, folder)
    model_dir = folder
    features, targets = input_output(model_dir)
    if index == 0:
        stacked_features = features
        stacked_targets = targets
    else:
        stacked_features = np.concatenate((stacked_features, features), axis=2)
        stacked_targets = np.concatenate((stacked_targets, targets), axis=2)

0 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim99
1 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim75
2 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim88
3 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim32
4 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim64
5 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim72
6 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim50
7 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim51
8 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim73
9 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim10
10 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim52
11 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim39
12 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim41
13 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim86
14 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simulations/sim65
15 /pscratch/sd/l/lijingw/Snodgrass_ATS_Model/simu

In [12]:
stacked_features.shape

(740, 42, 73000)

In [13]:
np.save("s_stacked_features.npy",stacked_features)
np.save("s_stacked_targets.npy",stacked_targets)

In [ ]:
#np.savez_compressed('inpput_output.npz', arr1=stacked_features, arr2=stacked_targets)